In [ ]:
# %pip install Jinja2
# %pip install imbalanced-learn
# %pip install --upgrade pypalettes
# %pip install shap
# %pip install lime
%pip show ipython

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split,RandomizedSearchCV,StratifiedKFold,GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import (accuracy_score, roc_auc_score, precision_score,
                             recall_score, f1_score, confusion_matrix,
                             ConfusionMatrixDisplay, classification_report, roc_curve,make_scorer,roc_auc_score)
from imblearn.over_sampling import SMOTE
from collections import Counter
import lime
import lime.lime_tabular
from lime import lime_text
from lime.lime_tabular import LimeTabularExplainer
from sklearn.svm import SVC
from sklearn import svm
from imblearn.pipeline import Pipeline
from scipy.stats import randint, uniform
from imblearn.over_sampling import SVMSMOTE
from sklearn.decomposition import PCA
from IPython.display import display

import shap
%matplotlib inline

# Load the dataset

In [ ]:
df = pd.read_csv("D:\Master\Completed Unit\ICT_606_Machine Learning_TSA24\Assignment 1_27 Sep 24\diabetes_data_upload.csv")
df

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

# 2.1.1 EDA
# A .Imbalanced Distribution of Target variable

In [ ]:
plt.figure(figsize=(10,7))
plt.subplot(121)
df["class"].value_counts().plot.pie(autopct = "%1.0f%%",colors = sns.color_palette("prism",7),startangle = 60,labels=["Positive","Negative"],
wedgeprops={"linewidth":2,"edgecolor":"k"},explode=[.1,0],shadow =True)
plt.title("Distribution of Target  Variable")

plt.subplot(122)
ax = df["class"].value_counts().plot(kind="barh")

for i,j in enumerate(df["class"].value_counts().values):
    ax.text(.7,i,j,weight = "bold",fontsize=20)

plt.title("Count of Target Variable")

# B .Gender Distribution on Positive Class

In [ ]:
ax=sns.countplot(df, x="Gender",hue='class')

for container in ax.containers:
    ax.bar_label(container)

plot_criteria= ['Gender', 'class']

cm = sns.light_palette("red", as_cmap=True)
(round(pd.crosstab(df[plot_criteria[0]], df[plot_criteria[1]], normalize='columns') * 100,2)).style.background_gradient(cmap = cm)

# C.Chi-Square Test

In [ ]:
#Map 'positive' to 1 and 'negative' to 0
df['class'] = df['class'].apply(lambda x: 0 if x=='Negative' else 1)
df['class'].head()

# drop class column
X = df.drop(['class'], axis=1)
y = df['class']

# Storing Features
objectList = X.select_dtypes(include = "object").columns
print(objectList)

In [ ]:
# Label Encoding for object to numeric conversion

le = LabelEncoder()

for feature in objectList:
    X[feature] = X[feature].astype('category').cat.codes

print (X.info())

In [ ]:
# Initialize DataFrame to store Chi-Square statistics or p-values
chi2_stats = pd.DataFrame(index=objectList, columns=['Chi2 Stat', 'P-value'])

# Perform Chi-Square test for each categorical feature
for feature in X.columns:
    # Create contingency table
    contingency_table = pd.crosstab(X[feature], y)

    # Perform Chi-Square test
    chi2_stat, p_value, dof, ex = chi2_contingency(contingency_table)

    # Store results
    chi2_stats.loc[feature, 'Chi2 Stat'] = chi2_stat
    chi2_stats.loc[feature, 'P-value'] = p_value

In [ ]:
corr_plot=X.corrwith(y)
print(corr_plot)

In [ ]:
corr_df = pd.DataFrame(corr_plot, columns=['Correlation'])

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(8, 4))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt='.2f')
plt.title('Correlation of Features with Target Variable')
plt.show()

In [ ]:
# Ensure data is numeric
chi2_stats['Chi2 Stat'] = pd.to_numeric(chi2_stats['Chi2 Stat'], errors='coerce')
chi2_stats['P-value'] = pd.to_numeric(chi2_stats['P-value'], errors='coerce')

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(chi2_stats[['P-value']], annot=True, cmap='coolwarm', fmt='.2f', vmin=0, vmax=1)
plt.title('P-Values Heatmap')
plt.xlabel('P-Value')
plt.ylabel('Features')
plt.show()

In [ ]:
age_stats = df.groupby('class')['Age'].describe()
age_stats['IQR'] = age_stats['75%'] - age_stats['25%']

# Create a boxplot
plt.figure(figsize=(5, 5))
boxplot = sns.boxplot(x='class', y='Age', data=df, palette='Set2')

# Overlay IQR with error bars
for i, class_label in enumerate(age_stats.index):
    q1 = df[df['class'] == class_label]['Age'].quantile(0.25)
    q3 = df[df['class'] == class_label]['Age'].quantile(0.75)
    print(q1)
    print(q3)
    iqr = q3 - q1
    plt.errorbar(x=i, y=age_stats.loc[class_label, 'mean'],
                 yerr=iqr / 2, fmt='o', color='red', capsize=5)


plt.title('Age Distribution by Class with IQR Highlights')
plt.xlabel('Class')
plt.ylabel('Age')
plt.show()

In [ ]:
# Histogram with KDE overlay
plt.figure(figsize=(8, 5))
sns.histplot(data=df, x='Age', hue='class', kde=True, palette='Set2', bins=15)
plt.title('Histogram and KDE of Age by Class')

# PCA

In [ ]:

X_1 = df.drop(['class'], axis=1)
y_1 = df['class']

In [ ]:
# Label Encoding for object to numeric conversion

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feature in objectList:
    X_1[feature] = X_1[feature].astype('category').cat.codes

print (X_1.info())

In [ ]:
# Data Scaling
x_1scaled = StandardScaler().fit_transform(X_1)

In [ ]:
# Reduce from 16 to 5 features with PCA
pca = PCA(n_components=5)

In [ ]:
# Fit and transform data
pca_features = pca.fit_transform(x_1scaled)

In [ ]:
# Bar plot of explained_variance
explained_variance = pca.explained_variance_
explained_variance_ratio = pca.explained_variance_ratio_ * 100

# Sort explained variance and ratios in descending order
sorted_indices = np.argsort(explained_variance_ratio)[::-1]  # Indices for sorting in descending order
sorted_explained_variance = explained_variance_ratio[sorted_indices]
sorted_cumulative_variance = np.cumsum(sorted_explained_variance)

# Create a scree plot
plt.figure(figsize=(6, 8))
plt.bar(range(1, len(sorted_explained_variance) + 1), sorted_explained_variance, alpha=0.6, color='b', label='Individual Explained Variance')

# Plot cumulative explained variance
plt.plot(range(1, len(sorted_explained_variance) + 1), sorted_cumulative_variance, color='red', marker='o', label='Cumulative Explained Variance')

# Annotate each cumulative point with its percentage
for i, cum_var in enumerate(sorted_cumulative_variance):
    plt.annotate(f'{cum_var:.1f}%',
                 xy=(i + 1, cum_var),
                 xytext=(i + 1, cum_var + 3),  # Adjust vertical position as needed
                 ha='center',
                 fontsize=9)




# Mark the elbow point (adjust this index based on your analysis)
elbow_index = 2 # Adjust based on where you visually determine the elbow is
plt.axvline(x=elbow_index, color='green', linestyle='--', label='Elbow Point')

# Labels and title
plt.xlabel('PCA Feature')
plt.ylabel('Explained Variance (%)')
plt.title('Scree Plot (Explained Variance by Principal Components)')
plt.xticks(range(1, len(explained_variance) + 1))
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Assuming feature_names is defined from the columns of X_1
feature_names = X_1.columns

def create_pca_biplot(X, y, feature_names, n_components=2, scale=True, **kwargs):
    """
    Generates a PCA biplot visualizing the first two principal components
    and the contribution of the original features, with enhanced visual clarity.

    Args:
        X (pd.DataFrame or np.ndarray): The feature data.
        y (pd.Series or np.ndarray): The target variable for coloring data points.
        feature_names (list): List of feature names corresponding to the columns of X.
        n_components (int, optional): Number of principal components to compute. Defaults to 2.
        scale (bool, optional): Whether to scale the data before applying PCA. Defaults to True.
        **kwargs: Additional keyword arguments passed to plt.scatter.
    """
    if scale:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = X

    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(X_scaled)
    explained_variance_ratio = pca.explained_variance_ratio_

    plt.figure(figsize=(10, 10))  # Increased figure size for better readability
    scatter = plt.scatter(pca_features[:, 0], pca_features[:, 1],
                            c=y, cmap=plt.cm.RdBu,  # More informative colormap
                            alpha=0.7, edgecolor='k', s=80, **kwargs) # Increased marker size and black edges

    # Add arrows for feature vectors with improved aesthetics and scaling
    feature_vectors = pca.components_.T
    scale_factor = max(np.abs(pca_features[:, :2]).flatten()) * 0.7  # Adjust scale for better fit

    for i, varname in enumerate(feature_names):
        x = feature_vectors[i, 0] * scale_factor
        y = feature_vectors[i, 1] * scale_factor
        plt.arrow(0, 0, x, y, color='forestgreen', alpha=0.8, linewidth=1.2, head_width=0.05 * scale_factor, head_length=0.1 * scale_factor)
        plt.text(x * 1.1, y * 1.1, varname, fontsize=12, color='dimgray', ha='center', va='center',
                 bbox=dict(facecolor='white', edgecolor='lightgray', boxstyle='round,pad=0.2'))

    # Add informative axis labels and title
    plt.xlabel(f'Principal Component 1 ({explained_variance_ratio[0]*100:.1f}%)', fontsize=14)
    plt.ylabel(f'Principal Component 2 ({explained_variance_ratio[1]*100:.1f}%)', fontsize=14)
    plt.title('PCA Biplot Visualizing Data and Feature Contributions', fontsize=16)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.axhline(y=0, color='k', linewidth=0.5)
    plt.axvline(x=0, color='k', linewidth=0.5)

    # Add a legend if a colormap is used for the target variable
    if 'c' in kwargs or (isinstance(y, (np.ndarray, list)) and len(np.unique(y)) > 1):
        legend = plt.legend(*scatter.legend_elements(), title="Target Class")
        plt.gca().add_artist(legend)

    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.show()


In [ ]:
# Call the improved biplot function
create_pca_biplot(X_1, y_1, feature_names)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =train_test_split(X, y,test_size=0.2,random_state=0,stratify=y)

# 2.1.2 RF Feature Selection

In [ ]:
# Train RandomForest Classifier
forest = RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train, y_train)

In [ ]:
df_1 = df.copy()

In [ ]:
# Create a new column with mapped numerical values using map
mapping = {'Positive': 1, 'Negative': 0}
df_1['class_num'] = df_1['class'].map(mapping)

In [ ]:
# Define feature columns (exclude the target variable 'class_num')
feature_columns = X.columns[X.columns != 'class_num']
X = df_1[feature_columns]  # Features for training
y = df_1['class_num']      #

In [ ]:
# Feature importances
importances = forest.feature_importances_
# Feature labels
feat_labels = feature_columns
# Sort the feature importances in descending order and get indices
indices = np.argsort(importances)[::-1]

# Print the feature importances
print("Feature importances:")
for f in range(X_train.shape[1]):
    print(f"{f + 1:2d}) {feat_labels[indices[f]]:<30} {importances[indices[f]]:.6f}")

plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]),importances[indices],align='center')
plt.xticks(range(X_train.shape[1]),
feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

# 2.2 Predicitve Model Construction

# Applying SMOTE

In [ ]:
smt = SMOTE()
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)
counter = Counter(y_train_sm)
print('After',counter)

In [ ]:
model = list()
resample = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()
Sensitivity =list()
Specificity = list()

def test_eval(clf_model, X_test, y_test, algo=None, sampling=None):

    # Test set prediction
    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    # Calculate metrics
    precision_val = precision_score(y_test, y_pred)
    recall_val = recall_score(y_test, y_pred)
    f1score_val = f1_score(y_test, y_pred)
    aucroc_val = roc_auc_score(y_test, y_prob[:,1])
    sensitivity_val = recall_val
    specificity_val = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Print metrics
    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print('AUC-ROC')
    print('='*60)
    print(roc_auc_score(y_test, y_prob[:,1]))
    print('='*60)
    print('Sensitivity (Recall)')
    print('=' * 60)
    print(sensitivity_val)
    print('=' * 60)
    print('Specificity')
    print('=' * 60)
    print(specificity_val)


    # Append metrics to lists
    model.append(algo)
    precision.append(precision_score(y_test,y_pred))
    recall.append(recall_score(y_test,y_pred))
    F1score.append(f1_score(y_test,y_pred))
    AUCROC.append(roc_auc_score(y_test, y_prob[:,1]))
    resample.append(sampling)
    Sensitivity.append(sensitivity_val)
    Specificity.append(specificity_val)

# Model 1 : Random Forest

In [ ]:
estimators = [2,10,30,50,100]
# Maximum number of depth in each tree:
max_depth = [i for i in range(5,16,2)]
# Minimum number of samples to consider to split a node:
min_samples_split = [2, 5, 10, 15, 20, 50, 100]
# Minimum number of samples to consider at each leaf node:
min_samples_leaf = [1, 2, 5]
# Including both Gini impurity and Entropy
criteria = ['gini', 'entropy']

cv = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)


clf_r = RandomForestClassifier(random_state=42)

# Define the parameter grid
rf_params = {
    'n_estimators': estimators,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'criterion': criteria  # Add criterion to the hyperparameter grid
}

In [ ]:
## Fitting the RF model
clf_RF = RandomizedSearchCV(clf_r, rf_params, cv=cv, scoring='accuracy', n_jobs=-1, n_iter=10, verbose=2)
clf_RF.fit(X_train_sm, y_train_sm)

In [ ]:
RF_SMOTE= clf_RF.best_estimator_

In [ ]:
#RF Evaluating the Model (After SMOTE):
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'smote')

# Model 2 : Logistic Regression

In [ ]:
#Feature Scaling
Scaler_X = StandardScaler()
X_train_lr = Scaler_X.fit_transform(X_train)
X_test_lr = Scaler_X.transform(X_test)

In [ ]:
# Logistic Regression

log_model=LogisticRegression()

params = {
    'C': np.logspace(-10, 1, 15),  # Regularization strength
    'class_weight': ['balanced'],  # Use balanced class weights
    'penalty': ['l1', 'l2'],  # Penalty types
    'solver': ['liblinear', 'lbfgs']  # Include solvers that support the penalties
}
cv = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)

# Create grid search using 5-fold cross validation
clf_LR = GridSearchCV(log_model, params, cv=cv, scoring='accuracy', n_jobs=-1)

In [ ]:
# Fit the LR model
clf_LR.fit(X_train_sm, y_train_sm)
LR_SMOTE=clf_LR.best_estimator_

In [ ]:
#LR Evaluating the Model (After SMOTE):
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'smote')

# Model 3 : SVC

In [ ]:
sc = StandardScaler().fit(X_train)
X_train_SVM = sc.transform(X_train)
X_test_SVM = sc.transform(X_test)

In [ ]:
# Initialize SVMSMOTE with the current k value
svm_smote = SVMSMOTE(k_neighbors=5, random_state=42)
svm_model = SVC(probability=True, random_state=42)
# Create a pipeline
pipeline = Pipeline([
    ('svmsmote', svm_smote),
    ('svc', svm_model)
])
# Resample the SVC model
X_train_SVM, y_train_SVM = svm_smote.fit_resample(X_train, y_train)

# Set up StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)

# Set up RandomizedSearchCV to find the best hyperparameters
param_grid = {
    'svc__C': [1, 10, 100, 1000],
    'svc__gamma': [1, 0.1, 0.001, 0.0001],
    'svc__kernel': ['linear', 'rbf']
}

In [ ]:
# Set up RandomizedSearchCV to find the best hyperparameters
clf_svm = RandomizedSearchCV(pipeline, param_distributions=param_grid,
                                    n_iter=10, cv=cv, scoring='accuracy',
                                    n_jobs=-1, verbose=2, random_state=42)

clf_svm.fit(X_train_SVM, y_train_SVM)
SVM_SMOTE=clf_svm.best_estimator_

In [ ]:
test_eval(clf_svm, X_test, y_test, 'SVC', 'smote')

# ROC Curve

In [ ]:
# Model 1 : RF
y_pre_prob_1 = RF_SMOTE.predict_proba(X_test)[:, 1]
fpr1, tpr1, thresholds1 = roc_curve(y_test, y_pre_prob_1)
auc1 = roc_auc_score(y_test, y_pre_prob_1)

# Model 2 : LR

y_pre_prob_2=LR_SMOTE.predict_proba(X_test)[:, 1]
fpr2, tpr2, thresholds2 = roc_curve(y_test, y_pre_prob_2)
auc2 = roc_auc_score(y_test, y_pre_prob_2)

# Model 3 : SVM

y_pre_prob_3=SVM_SMOTE.predict_proba(X_test)[:, 1]
fpr3, tpr3, thresholds3 = roc_curve(y_test, y_pre_prob_3)
auc3 = roc_auc_score(y_test, y_pre_prob_3)

plt.plot([0, 1], [0, 1], [0,1],'k--')
plt.plot(fpr1, tpr1, label="RF (AUC = %0.4f)" % auc1)
plt.plot(fpr2, tpr2, label="LR (AUC = %0.4f)" % auc2)
plt.plot(fpr3, tpr3, label="SVM (AUC = %0.4f)" % auc3)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend()
plt.show()

# 3.1 XAI - SHAP

In [ ]:
shap.initjs()

In [ ]:
# Create a Random Forest Classifier
rf_class = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_class.fit(X_train, y_train)

In [ ]:
# Create a KernelExplainer
explainer = shap.KernelExplainer(rf_class.predict, X_train)

# Explain the prediction using the KernelExplainer
shap_values = explainer.shap_values(X_test)

In [ ]:
# Plot the summary plot
shap.summary_plot(shap_values, X_test, show=False, feature_names=X.columns)
plt.show()

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")